# Introduction

Do higher film budgets lead to more box office revenue? Let's find out if there's a relationship using the movie budgets and financial performance data that I've scraped from [the-numbers.com](https://www.the-numbers.com/movie/budgets) on **May 1st, 2018**. 

<img src=https://i.imgur.com/kq7hrEh.png>

# Import Statements

In [64]:
import pandas as pd
import matplotlib.pyplot as plt


# Notebook Presentation

In [65]:
pd.set_option('display.float_format', '{:,.2f}'.format)

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Read the Data

In [66]:
data = pd.read_csv(r'Data_files/cost_revenue_dirty.csv')

# Explore and Clean the Data

**Challenge**: Answer these questions about the dataset:
1. How many rows and columns does the dataset contain?
2. Are there any NaN values present?
3. Are there any duplicate rows?
4. What are the data types of the columns?

In [67]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5391 entries, 0 to 5390
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Rank                   5391 non-null   int64 
 1   Release_Date           5391 non-null   object
 2   Movie_Title            5391 non-null   object
 3   USD_Production_Budget  5391 non-null   object
 4   USD_Worldwide_Gross    5391 non-null   object
 5   USD_Domestic_Gross     5391 non-null   object
dtypes: int64(1), object(5)
memory usage: 252.8+ KB


In [68]:
# print(type(data.loc[0, 'USD_Worldwide_Gross']))
data.head(3)

,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
0,5293,8/2/1915,The Birth of a Nation,"$110,000","$11,000,000","$10,000,000"
1,5140,5/9/1916,Intolerance,"$385,907",$0,$0
2,5230,12/24/1916,"20,000 Leagues Under the Sea","$200,000","$8,000,000","$8,000,000"


In [69]:
print(f'NaN values count: {data.isna().any(axis=1).sum()}')

NaN values count: 0


In [70]:
print(f'Duplicated values count: {data.duplicated().sum()}')
print(f'Duplicated titles count: {data.duplicated(subset='Movie_Title').sum()}')
print(f'Duplicated titles & dates count: {data.duplicated(subset=['Movie_Title', 'Release_Date']).sum()}')

Duplicated values count: 0
Duplicated titles count: 73
Duplicated titles & dates count: 1


### Show movies with the same titles

In [71]:
print(data[data.duplicated(subset='Movie_Title', keep=False)].shape)

data[data.duplicated(subset='Movie_Title', keep=False)].sort_values(by='Movie_Title')

(143, 6)


,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
2,5230,12/24/1916,"20,000 Leagues Under the Sea","$200,000","$8,000,000","$8,000,000"
64,3973,12/23/1954,"20,000 Leagues Under the Sea","$5,000,000","$28,200,000","$28,200,000"
3054,5289,12/31/2007,A Dog's Breakfast,"$120,000",$0,$0
3055,5290,12/31/2007,A Dog's Breakfast,"$120,000",$0,$0
393,4662,9/11/1984,A Nightmare on Elm Street,"$1,800,000","$25,504,513","$25,504,513"
...,...,...,...,...,...,...
1096,2427,6/3/1998,Twilight,"$20,000,000","$15,055,091","$15,055,091"
852,2638,2/23/1996,Unforgettable,"$18,000,000","$2,483,790","$2,483,790"
5342,3180,4/21/2017,Unforgettable,"$12,000,000","$15,668,012","$11,368,012"
3848,1321,2/18/2011,Unknown,"$40,000,000","$136,123,083","$63,686,397"


### Show duplicated movies (the same release date & title)

In [72]:
data[data.duplicated(subset=['Movie_Title', 'Release_Date'], keep=False)]

,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
3054,5289,12/31/2007,A Dog's Breakfast,"$120,000",$0,$0
3055,5290,12/31/2007,A Dog's Breakfast,"$120,000",$0,$0


### Data Type Conversions

**Challenge**: Convert the `Release_Date` column to a Pandas Datetime type. 

In [73]:
data['Release_Date'] = pd.to_datetime(data['Release_Date'])
data.dtypes

Rank                              int64
Release_Date             datetime64[ns]
Movie_Title                      object
USD_Production_Budget            object
USD_Worldwide_Gross              object
USD_Domestic_Gross               object
dtype: object

**Challenge**: Convert the `USD_Production_Budget`, `USD_Worldwide_Gross`, and `USD_Domestic_Gross` columns to a numeric format by removing `$` signs and `,`. 
<br>
<br>
Note that *domestic* in this context refers to the United States.

In [75]:
for (label, column) in data[['USD_Production_Budget', 'USD_Worldwide_Gross', 'USD_Domestic_Gross']].items():
    data[label] = data[label].str.replace('$', '')
    data[label] = data[label].str.replace(',', '')
    data[label] = data[label].astype(float)

data.dtypes

Rank                              int64
Release_Date             datetime64[ns]
Movie_Title                      object
USD_Production_Budget           float64
USD_Worldwide_Gross             float64
USD_Domestic_Gross              float64
dtype: object

In [78]:
data.head(3)

,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
0,5293,1915-08-02,The Birth of a Nation,"110,000.00","11,000,000.00","10,000,000.00"
1,5140,1916-05-09,Intolerance,"385,907.00",0.00,0.00
2,5230,1916-12-24,"20,000 Leagues Under the Sea","200,000.00","8,000,000.00","8,000,000.00"


### Descriptive Statistics

**Challenge**: 

1. What is the average production budget of the films in the data set?
2. What is the average worldwide gross revenue of films?
3. What were the minimums for worldwide and domestic revenue?
4. Are the bottom 25% of films actually profitable or do they lose money?
5. What are the highest production budget and highest worldwide gross revenue of any film?
6. How much revenue did the lowest and highest budget films make?

In [83]:
data.select_dtypes('number').describe()

,Rank,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
count,"5,391.00","5,391.00","5,391.00","5,391.00"
mean,"2,696.00","31,113,737.58","88,855,421.96","41,235,519.44"
std,"1,556.39","40,523,796.88","168,457,757.00","66,029,346.27"
min,1.00,"1,100.00",0.00,0.00
25%,"1,348.50","5,000,000.00","3,865,206.00","1,330,901.50"
50%,"2,696.00","17,000,000.00","27,450,453.00","17,192,205.00"
75%,"4,043.50","40,000,000.00","96,454,455.00","52,343,687.00"
max,"5,391.00","425,000,000.00","2,783,918,982.00","936,662,225.00"


### Search for Star Wars

In [107]:
data[data['Movie_Title'].str.contains('Star Wars')].sort_values(by='Release_Date')

,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
221,3239,1977-05-25,Star Wars Ep. IV: A New Hope,"11,000,000.00","786,598,007.00","460,998,007.00"
275,2196,1980-05-21,Star Wars Ep. V: The Empire Strikes Back,"23,000,000.00","534,171,960.00","290,271,960.00"
359,1657,1983-05-25,Star Wars Ep. VI: Return of the Jedi,"32,500,000.00","572,705,079.00","309,205,079.00"
1249,272,1999-05-19,Star Wars Ep. I: The Phantom Menace,"115,000,000.00","1,027,044,677.00","474,544,677.00"
1810,274,2002-05-16,Star Wars Ep. II: Attack of the Clones,"115,000,000.00","656,695,615.00","310,676,740.00"
2436,273,2005-05-19,Star Wars Ep. III: Revenge of the Sith,"115,000,000.00","848,998,877.00","380,270,577.00"
3202,3569,2008-08-15,Star Wars: The Clone Wars,"8,500,000.00","68,695,443.00","35,161,554.00"
5079,2,2015-12-18,Star Wars Ep. VII: The Force Awakens,"306,000,000.00","2,058,662,225.00","936,662,225.00"
5293,38,2016-12-16,Rogue One: A Star Wars Story,"200,000,000.00","1,050,988,488.00","532,177,324.00"


### str contains with nan values

In [108]:
import numpy as np
test_df = pd.DataFrame({'A': [11, 22, 33], 'B': ['Star afda gfg', 'dafafd', np.nan]})
test_df

,A,B
0,11,Star afda gfg
1,22,dafafd
2,33,NaN


In [101]:
print(f'{test_df['B'].str.contains('Star')}\n')
print(f'{test_df['B'].str.contains('Star', na=False)}')

0     True
1    False
2      NaN
Name: B, dtype: object

0     True
1    False
2    False
Name: B, dtype: bool


In [102]:
test_df[test_df['B'].str.contains('Star', na=False)]

,A,B
0,11,Star afda gfg


# Investigating the Zero Revenue Films

**Challenge** How many films grossed $0 domestically (i.e., in the United States)? What were the highest budget films that grossed nothing?

**Challenge**: How many films grossed $0 worldwide? What are the highest budget films that had no revenue internationally?

### Filtering on Multiple Conditions

**Challenge**: Use the [`.query()` function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html) to accomplish the same thing. Create a subset for international releases that had some worldwide gross revenue, but made zero revenue in the United States. 

Hint: This time you'll have to use the `and` keyword.

### Unreleased Films

**Challenge**:
* Identify which films were not released yet as of the time of data collection (May 1st, 2018).
* How many films are included in the dataset that have not yet had a chance to be screened in the box office? 
* Create another DataFrame called data_clean that does not include these films. 

In [ ]:
# Date of Data Collection
scrape_date = pd.Timestamp('2018-5-1')

### Films that Lost Money

**Challenge**: 
What is the percentage of films where the production costs exceeded the worldwide gross revenue? 

# Seaborn for Data Viz: Bubble Charts

### Plotting Movie Releases over Time

**Challenge**: Try to create the following Bubble Chart:

<img src=https://i.imgur.com/8fUn9T6.png>



# Converting Years to Decades Trick

**Challenge**: Create a column in `data_clean` that has the decade of the release. 

<img src=https://i.imgur.com/0VEfagw.png width=650> 

Here's how: 
1. Create a [`DatetimeIndex` object](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html) from the Release_Date column. 
2. Grab all the years from the `DatetimeIndex` object using the `.year` property.
<img src=https://i.imgur.com/5m06Ach.png width=650>
3. Use floor division `//` to convert the year data to the decades of the films.
4. Add the decades as a `Decade` column to the `data_clean` DataFrame.

### Separate the "old" (before 1969) and "New" (1970s onwards) Films

**Challenge**: Create two new DataFrames: `old_films` and `new_films`
* `old_films` should include all the films before 1969 (up to and including 1969)
* `new_films` should include all the films from 1970 onwards
* How many films were released prior to 1970?
* What was the most expensive film made prior to 1970?

# Seaborn Regression Plots

**Challenge**: Use Seaborn's `.regplot()` to show the scatter plot and linear regression line against the `new_films`. 
<br>
<br>
Style the chart

* Put the chart on a `'darkgrid'`.
* Set limits on the axes so that they don't show negative values.
* Label the axes on the plot "Revenue in \$ billions" and "Budget in \$ millions".
* Provide HEX colour codes for the plot and the regression line. Make the dots dark blue (#2f4b7c) and the line orange (#ff7c43).

Interpret the chart

* Do our data points for the new films align better or worse with the linear regression than for our older films?
* Roughly how much would a film with a budget of $150 million make according to the regression line?

# Run Your Own Regression with scikit-learn

$$ REV \hat ENUE = \theta _0 + \theta _1 BUDGET$$

**Challenge**: Run a linear regression for the `old_films`. Calculate the intercept, slope and r-squared. How much of the variance in movie revenue does the linear model explain in this case?

# Use Your Model to Make a Prediction

We just estimated the slope and intercept! Remember that our Linear Model has the following form:

$$ REV \hat ENUE = \theta _0 + \theta _1 BUDGET$$

**Challenge**:  How much global revenue does our model estimate for a film with a budget of $350 million? 